In [1]:
from pathlib import Path
from gammapy.data import EventList, GTI, DataStore
from astropy import units as u
from astropy.coordinates import SkyCoord
from gammapy.maps import  MapAxis, WcsGeom
from gammapy.datasets import  MapDataset
from gammapy.makers import MapDatasetMaker, SafeMaskMaker

In [2]:
import logging
import warnings

#now we will Create and configure logger 
logging.basicConfig(filename="std.log",
					format='%(asctime)s %(message)s', 
					filemode='w') 

#Let us Create an object 
logger=logging.getLogger()
#Now we are going to Set the threshold of logger to DEBUG 
logger.setLevel(logging.WARNING)
warnings.simplefilter('ignore')

In [3]:
# let's read all the event files into the DC folder:
cwd = Path.cwd()
path = cwd / 'new_events/new_events_diffuse'
paths = list(path.rglob("selected_events*.fits"))
new_path = cwd / 'new_events/new_events_diffuse+fermi'
#Here I create a new folder to store the selected events, and select all the events with MC_ID 10001
#(new_path).mkdir(parents = True)
for i, filename_1 in enumerate(paths):
    filenumber = "{0:04d}".format(i)
    filename_2 = f'new_events/new_events_fermi/selected_events_fermi_{filenumber}.fits'
    events_1 = EventList.read(filename_1)
    events_2 = EventList.read(filename_2)
    gti_1 = GTI.read(filename_1)
    gti_2 = GTI.read(filename_2)
    # stack in place, now the _1 object contains the information of both
    gti_1.stack(gti_2)
    events_1.stack(events_2)
    events_1.write(new_path / f"selected_events_diffuse+fermi_{filenumber}.fits", gti=gti_1, overwrite=True)

In [4]:
# let's read all the new selected event files into the "new_events" folder:
new_paths = list(new_path.rglob("selected_events*.fits"))
#let's store the files into a `DataStore` object
data_store = DataStore.from_events_files(events_paths=new_paths, 
                                         irfs_paths="../irfs/"+
                                         "astri_100_43_008_0502_C0_20_AVERAGE_50h_SC_v1.0.lv3.fits")
print(data_store)
# Let's save them on disk:
data_store.hdu_table.write("datastores/selected_diffuse+fermi_hdu-index.fits.gz", overwrite=True)
data_store.obs_table.write("datastores/selected_diffuse+fermi_obs-index.fits.gz", overwrite=True)
# Let's read the DataStore
data_store = DataStore.from_dir("./datastores", "selected_diffuse+fermi_hdu-index.fits.gz", "selected_diffuse+fermi_obs-index.fits.gz")
data_store.info()
observations = data_store.get_observations()
table = data_store.obs_table
pos_obs = SkyCoord(table['GLON_PNT'], table['GLAT_PNT'], frame='galactic', unit='deg')

Data store:
HDU index table:
BASE_DIR: .
Rows: 1080
OBS_ID: 0 -- 179
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_3gauss']


Observation table:
Observatory name: 'N/A'
Number of observations: 180

Data store:
HDU index table:
BASE_DIR: datastores
Rows: 1080
OBS_ID: 0 -- 179
HDU_TYPE: ['aeff', 'bkg', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'bkg_3d', 'edisp_2d', 'events', 'gti', 'psf_3gauss']


Observation table:
Observatory name: 'N/A'
Number of observations: 180



### Creation of a Skymap to have a look at the diffuse morphology

In [5]:
pos_target = SkyCoord(73.0, 2.0, frame='galactic', unit='deg')

energy_axis = MapAxis.from_bounds(
    0.7, 100, nbin=10, name="energy", unit="TeV", interp="log"
)
geom = WcsGeom.create(
    skydir=pos_target,
    axes=[energy_axis],
    width=[22 * u.deg, 10 * u.deg],
    binsz=0.2 * u.deg,
    frame="galactic",
)

stacked = MapDataset.create(geom=geom)
maker = MapDatasetMaker(selection=["counts", "background", "exposure", "edisp", "psf"])
maker_safe_mask = SafeMaskMaker(methods=["aeff-default"])

for obs in observations:
    cutout = stacked.cutout(obs.pointing_radec, width="12 deg")
    dataset = maker.run(cutout, obs)
    dataset = maker_safe_mask.run(dataset, obs)
    stacked.stack(dataset)   

In [6]:
datasets_path = cwd / 'datasets' / 'analysis_selected_diffuse+fermi+bkg_ids_width_22_10'
(datasets_path).mkdir(parents = True)
filename = datasets_path / "selected-diffuse+fermi+bkg-ids-dataset_width_22_10.fits.gz"
stacked.write(filename, overwrite=True)